# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [3]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [5]:
symbol = 'aapl'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': ') RONAGTEGAMSLNLSSLET/K (A CDBQAE', 'calculationPrice': 'close', 'open': 126.4, 'openTime': 1635258921330, 'openSource': 'liiafcfo', 'close': 123.64, 'closeTime': 1657470539171, 'closeSource': 'licfifoa', 'high': 128.315, 'highTime': 1628189739454, 'highSource': 'dc nedlpiaiee rtey51 mu', 'low': 121, 'lowTime': 1664911331340, 'lowSource': ' iude p 5tecidna1lyeerm', 'latestPrice': 126.44, 'latestSource': 'Close', 'latestTime': 'February 25, 2021', 'latestUpdate': 1682973825720, 'latestVolume': 149174694, 'iexRealtimePrice': 125.347, 'iexRealtimeSize': 104, 'iexLastUpdated': 1668068025141, 'delayedPrice': 125, 'delayedPriceTime': 1671599290591, 'oddLotDelayedPrice': 122.2, 'oddLotDelayedPriceTime': 1644051883143, 'extendedPrice': 124.43, 'extendedChange': -0.22, 'extendedChangePercent': -0.00182, 'extendedPriceTime': 1660063586498, 'previousClose': 126.53, 'previousVolume': 116122948, 'change': -4.41, 'changePercent': -0.

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [6]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

33.56

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns
            ),
            ignore_index = True
        )

df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,123.80,54.45,N/A
1,AAL,21.40,-1.18,N/A
2,AAP,161.93,23.5,N/A
3,AAPL,123.13,33.33,N/A
4,ABBV,112.19,39.68,N/A
...,...,...,...,...
500,YUM,107.40,37.01,N/A
501,ZBH,172.50,-248.13,N/A
502,ZBRA,514.60,55.47,N/A
503,ZION,55.54,24.09,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
df.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
df = df[df['Price-to-Earnings Ratio'] > 0]
df = df[:50]
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.990,2.32,N/A
1,BIO,583.330,4.65,N/A
2,AIV,5.000,4.95,N/A
3,ALL,113.040,6.32,N/A
4,UNM,27.640,7.22,N/A
5,EBAY,58.700,7.27,N/A
6,AFL,49.870,7.38,N/A
7,CPB,46.370,7.97,N/A
8,KIM,19.330,8.34,N/A
9,CE,145.140,8.5,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [10]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [11]:
position_size = float(input('Investment')) / len(df.index)

Investment10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [12]:
df['Number of Shares to Buy'] = df.apply(lambda x: math.floor(position_size / x['Price']),axis=1)
df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,37.990,2.32,5264
1,BIO,583.330,4.65,342
2,AIV,5.000,4.95,40000
3,ALL,113.040,6.32,1769
4,UNM,27.640,7.22,7235
5,EBAY,58.700,7.27,3407
6,AFL,49.870,7.38,4010
7,CPB,46.370,7.97,4313
8,KIM,19.330,8.34,10346
9,CE,145.140,8.5,1377


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [13]:
symbol = 'A'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
# print(data['AAPL']['advanced_stats']['enterpriseValue'])

# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value / gross_profit
print(ev_to_ebitda)

32.67496886912355


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [14]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    'N/A',
                    data[symbol]['quote']['peRatio'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToBook'],
                    'N/A',
                    data[symbol]['advanced-stats']['priceToSales'],
                    'N/A',
                    ev_to_ebitda,
                    'N/A',
                    ev_to_gross_profit,
                    'N/A',
                    'N/A'
                ],
                index=rv_columns
            ),
            ignore_index=True
        )

In [15]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.60,N/A,54.97,N/A,7.87,N/A,7.22,N/A,31.884541,N/A,13.347529,N/A,N/A
1,AAL,21.82,N/A,-1.14,N/A,-2.0,N/A,0.7849,N/A,-4.348140,N/A,2.197021,N/A,N/A
2,AAP,162.59,N/A,23.3,N/A,3.02,N/A,1.06,N/A,10.496562,N/A,2.369813,N/A,N/A
3,AAPL,121.57,N/A,33.1,N/A,30.89,N/A,6.97,N/A,25.265919,N/A,18.801874,N/A,N/A
4,ABBV,112.67,N/A,41.21,N/A,15.1,N/A,4.14,N/A,13.859789,N/A,9.035813,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.50,N/A,37.23,N/A,-4.04,N/A,5.73,N/A,24.071624,N/A,9.958430,N/A,N/A
501,ZBH,168.60,N/A,-246.78,N/A,2.9,N/A,4.88,N/A,31.083229,N/A,8.422217,N/A,N/A
502,ZBRA,499.70,N/A,53.28,N/A,12.81,N/A,6.12,N/A,35.218904,N/A,13.405760,N/A,N/A
503,ZION,56.01,N/A,23.38,N/A,1.27,N/A,3.05,N/A,13.557922,N/A,3.503958,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,241.260,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,246.390,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.8,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,43.680,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.920,N/A,14.33,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,56.040,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,32.770,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,178.300,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,2073.330,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
326,MYL,16.183,N/A,31.66,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [17]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [19]:
from scipy.stats import percentileofscore as score
metrics = {
    'Price-to-Earnings Ratio' : 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio' : 'PS Percentile',
    'EV/EBITDA' : 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.60,N/A,54.97,85.940594,7.87,73.663366,7.2200,80.594059,31.884541,87.524752,13.347529,78.613861,N/A
1,AAL,21.82,N/A,-1.14,15.049505,-2.00,4.950495,0.7849,6.930693,-4.348140,3.366337,2.197021,9.50495,N/A
2,AAP,162.59,N/A,23.30,46.732673,3.02,41.584158,1.0600,11.980198,10.496562,20.0,2.369813,10.891089,N/A
3,AAPL,121.57,N/A,33.10,66.336634,30.89,95.643564,6.9700,80.0,25.265919,79.207921,18.801874,92.277228,N/A
4,ABBV,112.67,N/A,41.21,77.029703,15.10,88.514851,4.1400,60.09901,13.859789,38.415842,9.035813,57.227723,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.50,N/A,37.23,72.475248,-4.04,4.752475,5.7300,73.465347,24.071624,76.039604,9.958430,66.930693,N/A
501,ZBH,168.60,N/A,-246.78,1.188119,2.90,39.80198,4.8800,68.811881,31.083229,86.336634,8.422217,54.059406,N/A
502,ZBRA,499.70,N/A,53.28,84.950495,12.81,85.742574,6.1200,75.247525,35.218904,88.712871,13.405760,78.811881,N/A
503,ZION,56.01,N/A,23.38,47.128713,1.27,13.564356,3.0500,45.346535,13.557922,37.029703,3.503958,17.227723,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [21]:
from statistics import mean

rv_dataframe['RV Score'] = rv_dataframe.apply(
    lambda x: mean(
        [
            x['PE Percentile'],
            x['PB Percentile'],
            x['PS Percentile'],
            x['EV/EBITDA Percentile'],
            x['EV/GP Percentile']
        ]
    ),
    axis=1
)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,127.60,N/A,54.97,85.940594,7.87,73.663366,7.2200,80.594059,31.884541,87.524752,13.347529,78.613861,81.267327
1,AAL,21.82,N/A,-1.14,15.049505,-2.00,4.950495,0.7849,6.930693,-4.348140,3.366337,2.197021,9.50495,7.960396
2,AAP,162.59,N/A,23.30,46.732673,3.02,41.584158,1.0600,11.980198,10.496562,20.0,2.369813,10.891089,26.237624
3,AAPL,121.57,N/A,33.10,66.336634,30.89,95.643564,6.9700,80.0,25.265919,79.207921,18.801874,92.277228,82.693069
4,ABBV,112.67,N/A,41.21,77.029703,15.10,88.514851,4.1400,60.09901,13.859789,38.415842,9.035813,57.227723,64.257426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,109.50,N/A,37.23,72.475248,-4.04,4.752475,5.7300,73.465347,24.071624,76.039604,9.958430,66.930693,58.732673
501,ZBH,168.60,N/A,-246.78,1.188119,2.90,39.80198,4.8800,68.811881,31.083229,86.336634,8.422217,54.059406,50.039604
502,ZBRA,499.70,N/A,53.28,84.950495,12.81,85.742574,6.1200,75.247525,35.218904,88.712871,13.405760,78.811881,82.693069
503,ZION,56.01,N/A,23.38,47.128713,1.27,13.564356,3.0500,45.346535,13.557922,37.029703,3.503958,17.227723,32.059406


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [23]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

<ipython-input-23-e1a7b46c451c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.99,N/A,7.09,16.633663,0.5168,5.544554,0.4405,2.376238,3.487893,4.158416,0.417463,0.39604,5.821782
1,MCK,182.16,N/A,-7.21,10.891089,-58.2000,0.990099,0.1205,0.594059,6.872239,6.732673,2.658295,11.485149,6.138614
2,FTI,8.90,N/A,-0.68,15.643564,0.9380,7.722772,0.2968,0.990099,2.500424,3.960396,1.610731,5.742574,6.811881
3,AIG,44.43,N/A,-6.50,11.683168,0.6029,5.940594,0.9020,8.118812,7.125501,7.722772,0.879560,2.178218,7.128713
4,MET,60.60,N/A,10.32,19.009901,0.7016,6.138614,0.7852,7.128713,4.933167,4.752475,0.756202,1.782178,7.762376
5,ALL,113.36,N/A,6.35,16.435644,1.1800,11.386139,0.7534,6.534653,2.345937,3.762376,0.746105,1.584158,7.940594
6,AAL,21.82,N/A,-1.14,15.049505,-2.0000,4.950495,0.7849,6.930693,-4.348140,3.366337,2.197021,9.50495,7.960396
7,DXC,25.42,N/A,-2.30,13.861386,1.1300,10.693069,0.3610,1.782178,4.956279,4.950495,2.240734,10.09901,8.277228
8,F,11.93,N/A,-36.84,5.148515,1.5000,17.920792,0.3752,1.980198,6.041236,5.346535,2.719617,12.277228,8.534653
9,ABC,106.15,N/A,-6.61,11.287129,-31.3000,1.386139,0.1132,0.39604,7.908221,10.891089,3.694957,18.811881,8.554455


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [25]:
rv_dataframe['Number of Shares to Buy'] = rv_dataframe.apply(
    lambda x: math.floor(position_size / x['Price']), axis=1
)

In [26]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.99,7145,7.09,16.633663,0.5168,5.544554,0.4405,2.376238,3.487893,4.158416,0.417463,0.39604,5.821782
1,MCK,182.16,1097,-7.21,10.891089,-58.2000,0.990099,0.1205,0.594059,6.872239,6.732673,2.658295,11.485149,6.138614
2,FTI,8.90,22471,-0.68,15.643564,0.9380,7.722772,0.2968,0.990099,2.500424,3.960396,1.610731,5.742574,6.811881
3,AIG,44.43,4501,-6.50,11.683168,0.6029,5.940594,0.9020,8.118812,7.125501,7.722772,0.879560,2.178218,7.128713
4,MET,60.60,3300,10.32,19.009901,0.7016,6.138614,0.7852,7.128713,4.933167,4.752475,0.756202,1.782178,7.762376
5,ALL,113.36,1764,6.35,16.435644,1.1800,11.386139,0.7534,6.534653,2.345937,3.762376,0.746105,1.584158,7.940594
6,AAL,21.82,9165,-1.14,15.049505,-2.0000,4.950495,0.7849,6.930693,-4.348140,3.366337,2.197021,9.50495,7.960396
7,DXC,25.42,7867,-2.30,13.861386,1.1300,10.693069,0.3610,1.782178,4.956279,4.950495,2.240734,10.09901,8.277228
8,F,11.93,16764,-36.84,5.148515,1.5000,17.920792,0.3752,1.980198,6.041236,5.346535,2.719617,12.277228,8.534653
9,ABC,106.15,1884,-6.61,11.287129,-31.3000,1.386139,0.1132,0.39604,7.908221,10.891089,3.694957,18.811881,8.554455


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [27]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value-Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [28]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [29]:
writer.save()